In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math

In [ ]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-None.change_regress.epoch-8.pth.txt")

df

In [ ]:
pd.DataFrame(df[df["parameter"] != "original"][["img", "styles_score", "technical_score", "composition_score", "score"]].groupby("img").max().to_records()).sort_values("img")

In [ ]:
df[df["parameter"] == "original"][["img", "styles_score", "technical_score", "composition_score", "score"]].sort_values("img").reset_index()

In [ ]:
# & (df["parameter"] != "pixelate") & (df["parameter"] != "impulse_noise")
scores = pd.DataFrame(df[(df["parameter"] != "original")][["img", "styles_score", "technical_score", "composition_score"]].groupby("img").max().to_records()).sort_values("img").rename(columns={"styles_score":"best_score_styles", "technical_score":"best_score_technical", "composition_score":"best_score_composition"})
scores["orig_score_styles"] = df[df["parameter"] == "original"][["img", "styles_score"]].sort_values("img").reset_index()["styles_score"]
scores["orig_score_technical"] = df[df["parameter"] == "original"][["img", "technical_score"]].sort_values("img").reset_index()["technical_score"]
scores["orig_score_composition"] = df[df["parameter"] == "original"][["img", "composition_score"]].sort_values("img").reset_index()["composition_score"]
scores["diff_styles"] = scores["orig_score_styles"] - scores["best_score_styles"]
scores["diff_technical"] = scores["orig_score_technical"] - scores["best_score_technical"]
scores["diff_composition"] = scores["orig_score_composition"] - scores["best_score_composition"]
scores

In [ ]:
scores.nlargest(10, "diff_styles")

In [ ]:
scores.nlargest(10, "diff_technical")

In [ ]:
scores.nlargest(10, "diff_composition")

In [ ]:
scores.nsmallest(10, "diff_styles")

In [ ]:
scores.nsmallest(10, "diff_technical")

In [ ]:
scores.nsmallest(10, "diff_composition")

In [ ]:
nsmall = []
for i, row in scores.nsmallest(10, "diff_styles").iterrows():
    nsmall.append(df[(df["img"] == row.img) & (df["styles_score"] == row.best_score_styles)])
pd.concat(nsmall)

In [ ]:
nsmall = []
for i, row in scores.nsmallest(10, "diff_technical").iterrows():
    nsmall.append(df[(df["img"] == row.img) & (df["technical_score"] == row.best_score_technical)])
pd.concat(nsmall)

In [ ]:
nsmall = []
for i, row in scores.nsmallest(10, "diff_composition").iterrows():
    nsmall.append(df[(df["img"] == row.img) & (df["composition_score"] == row.best_score_composition)])
pd.concat(nsmall)

In [ ]:
sns.distplot(df[df["parameter"] == "original"]["score"], label="original")
sns.distplot(df[df["parameter"] != "original"]["score"], label="rest")
plt.legend()

In [ ]:
sns.distplot(df[df["parameter"] == "original"]["styles_score"], label="original")
sns.distplot(df[df["parameter"] != "original"]["styles_score"], label="rest")
plt.legend()

In [ ]:
sns.distplot(df[df["parameter"] == "original"]["technical_score"], label="original")
sns.distplot(df[df["parameter"] != "original"]["technical_score"], label="rest")
plt.legend()

In [ ]:
sns.distplot(df[df["parameter"] == "original"]["composition_score"], label="original")
sns.distplot(df[df["parameter"] != "original"]["composition_score"], label="rest")
plt.legend()

In [ ]:
def histogram_distortion(distortion:str, score:str):
    sns.distplot(df[df["parameter"] == "original"][score], label="original")
    for change in (val for val in mapping["all_changes"] if distortion in val):
        parameter, change = change.split(";")
        sns.distplot(df[(df["parameter"] == parameter) & (df["change"] == float(change))][score], label=f"{parameter}: {change}")
    plt.legend()

In [ ]:
def violin_distortion(distortion:str, score:str):
    plot_frame = df[(df["parameter"] == distortion) | (df["parameter"] == "original")]
    if distortion in parameter_range:
        plot_frame.loc[plot_frame["parameter"] == "original", "change"] = parameter_range[distortion]["default"]
    sns.violinplot(data=plot_frame, x="change", y=score, color="steelblue")

In [ ]:
histogram_distortion("brightness", "styles_score")

In [ ]:
histogram_distortion("jpeg_compression", "technical_score")

In [ ]:
histogram_distortion("vcrop", "composition_score")

In [ ]:
violin_distortion("contrast", "styles_score")

In [ ]:
violin_distortion("jpeg_compression", "technical_score")

In [ ]:
violin_distortion("hcrop", "composition_score")

In [ ]:
avg_scores = df[["parameter","change", "score", "styles_score", "technical_score", "composition_score"]].groupby(["parameter", "change"]).mean().reset_index()
avg_scores.to_csv("analysis/SSMTIA/avg_scores.csv", index=False)
avg_scores

In [ ]:
parameter = "saturation"
sns.lineplot(data=avg_scores[avg_scores.parameter == parameter], x="change", y="score", label="score")
sns.lineplot(data=avg_scores[avg_scores.parameter == parameter], x="change", y="styles_score", label="style")
sns.lineplot(data=avg_scores[avg_scores.parameter == parameter], x="change", y="technical_score", label="technical")
sns.lineplot(data=avg_scores[avg_scores.parameter == parameter], x="change", y="composition_score", label="composition")

In [ ]:
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

fg = sns.FacetGrid(df[(df["parameter"] == "saturation") & (df["change"] == 1.0)], row="pred_change", aspect=15, height=.5)
fg.map(sns.kdeplot, "pred_change_degree")

#fg.map(label, "pred_change")

#fg.set_titles("")
#fg.set(yticks=[])
#fg.despine(bottom=True, left=True)

In [ ]:
def calculate_spearman(distortion: str, polarity: str, img_names=df["img"].unique()):
    corr_l = []
    p_l = []
    original_df = df[df["parameter"] == "original"]
    if distortion in parameter_range:
        original_df["change"] = parameter_range[distortion]["default"]

    parameter_df = df[df["parameter"] == distortion]
    corr_df = pd.concat([parameter_df, original_df])[["img", "parameter", "change", "score"]]

    for i, f in enumerate(img_names):
        if i % 1000 == 0:
            print(i)
        corr_df_img = corr_df[corr_df["img"] == f]

        if distortion in parameter_range:
            default = parameter_range[distortion]["default"]
        else:
            default = 0

        if polarity == "pos":
            corr_df_img = corr_df_img[corr_df_img["change"] >= default]
        else:
            corr_df_img = corr_df_img[corr_df_img["change"] <= default]

        corr_df_img["change"] = corr_df_img["change"].apply(lambda x: abs((x) - (default)))

        c, p = stats.spearmanr(corr_df_img["score"], corr_df_img["change"])
        if math.isnan(c) or math.isnan(p):
            continue
        corr_l.append(c)
        p_l.append(p)
    return sum(corr_l) / len(corr_l), sum(p_l) / len(p_l)

In [ ]:
calculate_spearman(distortion="brightness", polarity="neg")